<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Task Solving with Provided Tools as Functions

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to make function calls with the new feature of OpenAI models (in model version 0613). A specified prompt and function configs must be passed to `AssistantAgent` to initialize the agent. The corresponding functions must be passed to `UserProxyAgent`, which will execute any function calls made by `AssistantAgent`. Besides this requirement of matching descriptions with functions, we recommend checking the system message in the `AssistantAgent` to ensure the instructions align with the function call descriptions.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [mathchat] option since we will import functions from `MathUserProxyAgent`:
```bash
pip install "pyautogen[mathchat]"
```

In [1]:
# %pip install "pyautogen[mathchat]~=0.1.0"

## Set your API Endpoint

The [`config_list_from_models`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_models) function tries to create a list of configurations using Azure OpenAI endpoints and OpenAI endpoints for the provided list of models. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.
If you open this notebook in google colab, you can upload your files by clicking the file icon on the left panel and then choosing "upload file" icon.

The following code excludes Azure OpenAI endpoints from the config list because some endpoints don't support functions yet. Remove the `exclude` argument if they do.

In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4"],
    },
)

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-3.5-turbo
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-3.5-turbo-16k
]
```


## Making Async and Sync Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. With the default system prompt of `AssistantAgent`, we allow the LLM assistant to perform tasks with code, and the `UserProxyAgent` would extract code blocks from the LLM response and execute them. With the new "function_call" feature, we define functions and specify the description of the function in the OpenAI config for the `AssistantAgent`. Then we register the functions in `UserProxyAgent`.

In [4]:
# define functions according to the function description
import time

llm_config = {
    "config_list": config_list,
}

coder = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="A proxy for the user for executing code.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)

from typing_extensions import Annotated

# An example async function
@user_proxy.register_for_execution()
@coder.register_for_llm(description="create a timer for N seconds")
async def timer(num_seconds: Annotated[str, "Number of seconds in the timer."]) -> str:
    for i in range(int(num_seconds)):
        time.sleep(1)
        # should print to stdout
    return "Timer is done!"


# An example sync function
@user_proxy.register_for_execution()
@coder.register_for_llm(description="create a stopwatch for N seconds")
def stopwatch(num_seconds: Annotated[str, "Number of seconds in the stopwatch."]) -> str:
    for i in range(int(num_seconds)):
        time.sleep(1)
    return "Stopwatch is done!"


# start the conversation
# 'await' is used to pause and resume code execution for async IO operations.
# Without 'await', an async function returns a coroutine object but doesn't execute the function.
# With 'await', the async function is executed and the current function is paused until the awaited function returns a result.
await user_proxy.a_initiate_chat(
    coder,
    message="Create a timer for 5 seconds and then a stopwatch for 5 seconds.",
)

user_proxy (to chatbot):

Create a timer for 5 seconds and then a stopwatch for 5 seconds.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: timer *****
Arguments: 
{"num_seconds":"5"}
******************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION timer...
user_proxy (to chatbot):

***** Response from calling function "timer" *****
Timer is done!
**************************************************

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: stopwatch *****
Arguments: 
{"num_seconds":"5"}
**********************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION stopwatch...
user_proxy (to chatbot):

***** Response from calling fu

# Async Function Call with Group Chat
Sync and async can be used in topologies beyond two agents. Below, we show this feature for a group chat.

In [5]:
markdownagent = autogen.AssistantAgent(
    name="Markdown_agent",
    system_message="Respond in markdown only",
    llm_config=llm_config,
)

# Add a function for robust group chat termination
@user_proxy.register_for_execution()
@markdownagent.register_for_llm()
@coder.register_for_llm(description="terminate the group chat")
def terminate_group_chat(message: Annotated[str, "Message to be sent to the group chat."]) -> str:
    return f"[GROUPCHAT_TERMINATE] {message}"


groupchat = autogen.GroupChat(agents=[user_proxy, coder, markdownagent], messages=[], max_round=12)
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=lambda x: "GROUPCHAT_TERMINATE" in x.get("content", ""),
)

In [6]:
await user_proxy.a_initiate_chat(
    manager,
    message="""
1) Create a timer for 5 seconds.
2) a stopwatch for 5 seconds.
3) Pretty print the result as md.
4) when 1-3 are done, terminate the group chat""",
)

user_proxy (to chat_manager):


1) Create a timer for 5 seconds.
2) a stopwatch for 5 seconds.
3) Pretty print the result as md.
4) when 1-3 are done, terminate the group chat

--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
chatbot (to chat_manager):

***** Suggested function Call: timer *****
Arguments: 
{"num_seconds":"5"}
******************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION timer...
user_proxy (to chat_manager):

***** Response from calling function "timer" *****
Timer is done!
**************************************************

--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
chatbot (to 